In [2]:
import re
import nltk
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

In [3]:
import PyPDF2

In [4]:
def extract(pdf_file):
    with open(pdf_file, 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf_file, strict=False)
        pdf_text = []

        for page in reader.pages:
            content = page.extract_text()
            pdf_text.append(content)

        return " ".join(pdf_text)

In [5]:
myresume = [extract('resumeIA copy.pdf')]

In [6]:
data = pd.read_csv('jobs.csv')
display(data.head(10))

,Position,Company,Link,Description
0,Technical Content Developer / writer,Korber Supply Chain,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,K√∂rber is the home for passionate people who ...
1,Software Engineer,Vodafone,https://uk.indeed.com/applystart?jk=df516aec25...,Software Engineer \n\n\nLocation: Blended / ho...
2,Software Support Lead,Acacium Group,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,CHS Healthcare (part of Acacium Group)\n Softw...
3,Backend Software Engineer,Enfuce,https://uk.indeed.com/applystart?jk=cef2e4b90b...,"ABOUT US\n Founded in 2016, Enfuce has become ..."
4,Graduate Software Developer,Townhouse,https://uk.indeed.com/rc/clk?jk=511e448b6c4c01...,About Townhouse\n Townhouse was created with a...
5,Development Manager (Permanent Home Working) -...,CET Careers,https://uk.indeed.com/applystart?jk=616fbee4b4...,"Working for CET is many things ‚Äì exciting, c..."
6,Software Engineer - Backend - Python,Stuart,https://uk.indeed.com/applystart?jk=420bcf1373...,Stuart (DPD Group) is a sustainable last-mile ...
7,Software Engineer,WEX Europe Services Ltd,https://uk.indeed.com/company/WEX-Travel/jobs/...,We‚Äôre the International Fleet Development Te...
8,Software QA Engineer 2,WEX Inc.,https://uk.indeed.com/applystart?jk=815040b84a...,About the Role\n As a Software Developer you...
9,Software Engineer - Backend,Aforza,https://uk.indeed.com/applystart?jk=a5dd4ee97c...,"¬£35,000 - ¬£45,000\nCardiff / Remote\n Compan..."


In [7]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Position     3000 non-null   object
 1   Company      3000 non-null   object
 2   Link         3000 non-null   object
 3   Description  3000 non-null   object
dtypes: object(4)
memory usage: 93.9+ KB


None

In [8]:
duplicates = []
dup_bool = list(data['Description'].duplicated())
for i in range(len(dup_bool)):
    if dup_bool[i]:
        duplicates.append(i)

In [9]:
duplicates

[15,
 18,
 31,
 33,
 42,
 58,
 60,
 62,
 105,
 106,
 120,
 121,
 126,
 136,
 149,
 150,
 151,
 153,
 155,
 165,
 169,
 174,
 181,
 182,
 185,
 186,
 193,
 197,
 221,
 227,
 241,
 242,
 245,
 260,
 262,
 264,
 268,
 270,
 272,
 273,
 279,
 285,
 286,
 300,
 304,
 310,
 319,
 321,
 324,
 326,
 332,
 334,
 335,
 336,
 338,
 339,
 342,
 345,
 346,
 351,
 352,
 359,
 360,
 361,
 362,
 363,
 365,
 375,
 379,
 380,
 398,
 402,
 407,
 408,
 413,
 415,
 417,
 420,
 426,
 427,
 433,
 441,
 446,
 453,
 454,
 456,
 458,
 465,
 467,
 468,
 488,
 494,
 495,
 496,
 497,
 498,
 499,
 512,
 525,
 526,
 529,
 534,
 551,
 558,
 579,
 600,
 630,
 660,
 661,
 665,
 669,
 675,
 678,
 680,
 690,
 691,
 708,
 709,
 711,
 715,
 716,
 717,
 722,
 730,
 732,
 736,
 739,
 747,
 750,
 763,
 764,
 769,
 770,
 771,
 774,
 778,
 780,
 782,
 785,
 789,
 802,
 804,
 805,
 808,
 809,
 811,
 812,
 814,
 816,
 820,
 825,
 830,
 834,
 842,
 846,
 855,
 856,
 858,
 862,
 869,
 870,
 871,
 874,
 875,
 881,
 885,
 888,
 893,


In [10]:
data = data.drop(index=duplicates)
data = data.reset_index(drop=True)
initial_data = data
data

,Position,Company,Link,Description
0,Technical Content Developer / writer,Korber Supply Chain,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,K√∂rber is the home for passionate people who ...
1,Software Engineer,Vodafone,https://uk.indeed.com/applystart?jk=df516aec25...,Software Engineer \n\n\nLocation: Blended / ho...
2,Software Support Lead,Acacium Group,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,CHS Healthcare (part of Acacium Group)\n Softw...
3,Backend Software Engineer,Enfuce,https://uk.indeed.com/applystart?jk=cef2e4b90b...,"ABOUT US\n Founded in 2016, Enfuce has become ..."
4,Graduate Software Developer,Townhouse,https://uk.indeed.com/rc/clk?jk=511e448b6c4c01...,About Townhouse\n Townhouse was created with a...
...,...,...,...,...
2408,Finance Analyst,PureGym Limited,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,The PureGym Group is the UKs largest gym opera...
2409,Data Protection Officer,BCT Resourcing,https://uk.indeed.com/applystart?jk=a6b9dec315...,We are looking for a Data Protection Officer t...
2410,P2P and Master Data Manager,Robert Half,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Robert Half is partnering exclusively with one...
2411,Ecommerce Product Data Administrator Apprentice,QA Apprenticeships,https://uk.indeed.com/applystart?jk=975e1e47c6...,Employer description: ZLT Electrical are a fas...


In [11]:
len(data['Description'][4])

2903

In [12]:
resumes = pd.read_csv('resumes.csv')
initial_resumes = resumes
display(resumes.head())

,Text
0,SUMMARY\n\n\n\t7 years of experience in Inform...
1,SUMMARY\n\n\n\tOver ten years of experience in...
2,SUMMARY\n\n\n\tHaving 7+ years of IT industry ...
3,SUMMARY\n\n\n\t8 years experience in analyzing...
4,SUMMARY\n\n\n\tA Microsoft Certified Senior .N...


In [13]:
resumes. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 0 to 686
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    687 non-null    object
dtypes: object(1)
memory usage: 5.5+ KB


In [14]:
resumes.duplicated().sum()

0

In [15]:
def clear_text(text):
    ind = text.find('enefit')
    if ind<200:
        ind = text.find('enefit',200)
    clear_text = text[:ind-1] if ind!=-1 else text
    clear_text = re.sub('</\[^>*]>?&$@!¬∑Ääìô', ' ', clear_text)
    clear_text = clear_text.lower()
    replacables=["äôre","äôs","äôll","äôd","äôt","äô","äò","‚",",","?",":","(",")"]
    for i in replacables:
        clear_text = clear_text.replace(i,"")
    return ' '.join(clear_text.split())

In [16]:
def remove_benefit(text):
    ind = text.find('enefit')
    if ind<200:
        ind = text.find('enefit',200)
    return text[:ind-1] if ind!=-1 else text

In [17]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    try:
        while True:
            word_list.remove(".")
    except ValueError:
        pass
    return ' '.join([lemmatizer.lemmatize(w) for w in word_list])

[nltk_data] Downloading package wordnet to /Users/akhat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
def modify_text(data):
    corpus = list(data)
    for i in tqdm(range(len(corpus))):
        corpus[i] = lemmatize(clear_text(corpus[i]))
    return corpus

In [19]:
datas = data['Description']

In [20]:
%%time
corpus = modify_text(datas)

  0%|          | 0/2413 [00:00<?, ?it/s]

CPU times: user 7.87 s, sys: 205 ms, total: 8.08 s
Wall time: 8.15 s


In [21]:
# nltk.download('stopwords')
# stopwords = set(nltk_stopwords.words('english'))

# count = CountVectorizer(stop_words=stopwords, ngram_range = (1,1))

# vector_data =  count.fit_transform(corpus)

In [22]:
# nltk.download('omw-1.4')

In [23]:
# 'benefi'.find('benefit',2)

In [24]:
corpus[1]

'software engineer location blended / home working salary competitive + vodafone benefit working pattern term time only term time working week = 41.6 i.e 41 week and 3 day with annual leave = 168 hour approx 22.4 day non-working week = 10.4 i.e 10 week and 2 day to be taken during school holiday this role is ideal for someone who is looking to juggle work life & home life if you have a young family caring responsibility or kid that are currently attending school please find more information below term time working we are excited to be offering term time work schedule in our software engineering team to support with our mission to create the future term time working is part of our flexible working policy and offer employee the opportunity to have time off during the school holiday vodafone is an inclusive employer and we recognise the importance of work-life balance and establishing new routine to support our employee in balancing work with personal commitment blended vodafone uk blende

In [25]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count = TfidfVectorizer(stop_words=list(stopwords), ngram_range=(1,1))
vector_data = count.fit_transform(corpus)

[nltk_data] Downloading package stopwords to /Users/akhat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
vector_data = vector_data.toarray()

In [27]:
vector_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
vector_data.shape

(2413, 18223)

In [29]:
def cosine_similarity(x,y):
    return 1 - spatial.distance.cosine(x,y)

In [30]:
resumes = resumes['Text']

In [31]:
resume_corpus = modify_text(resumes)

  0%|          | 0/687 [00:00<?, ?it/s]

In [32]:
resumes_vector = count.transform(resume_corpus)

In [33]:
resumes_vector = resumes_vector.toarray()

In [34]:
resumes_vector[0]

array([0., 0., 0., ..., 0., 0., 0.])

# Write here any index of resume

In [35]:
resume_index = 10

In [36]:
result = []
max_distance = 0
max_index = 0

for i in tqdm(range(len(vector_data))):
    result.append(cosine_similarity(resumes_vector[resume_index], vector_data[i]))

  0%|          | 0/2413 [00:00<?, ?it/s]

In [37]:
p = sorted(result, reverse = True)
ct = 0
for i in p:
    if i>0.1:
        ct+=1
    else: break
ct

335

In [38]:
max_distance

0

In [39]:
data[2500:]

,Position,Company,Link,Description


In [40]:
print('This is job offer')
initial_data.loc[max_index ]['Description']

This is job offer


'K√∂rber is the home for passionate people who innovate, collaborate, and love what they do. The entrepreneurial spirit is our joint DNA. We develop future technologies and support talents to deploy their skills and reach their full potential. We combine our strong heritage with our understanding of local specifics and a global mindset to bring people together and create valuable solutions for our customers. We aim for being the first to do the right thing at the right time.\nJoin the home for entrepreneurs!\nYour role in our team\n\nDevelop technical eLearning training, courses, and enablement materials for K√∂rber supply chain solutions\nDesign, document, and integrate concepts and materials for instructional environment\nPrepare programming exercises for classroom training\nInteract with content consumers to incorporate enhancements to training materials\nConfigure, install, and maintain software applications for creating virtual labs\nInstall and maintain other software and hardwar

In [41]:
max_index

0

In [42]:
print('This is resume')
initial_resumes.loc[resume_index]['Text']

This is resume


'SUMMARY\n\n\n\tSoftware professional with 8 years of experience in designing Windows and Web based applications.\n\tInvolved in all stages of SDLC such as Requirement gathering, Design, Development and Testing.\n\tExperience in working with both Waterfall and Agile development methodology.\n\tExpertise in developing various applications using Visual Studio .NET technologies like C#, ASP.NET, ADO.NET, XML, Windows Services, Web Services, WPF,WCF, MVC.\n\tExperience in developing front - end User Interface using Angular JS, WPF, HTML and CSS.\n\tExtensive experience in Client side using JavaScript, JQuery.\n\tExtensively worked on ADO.NET to access SQL server 2008/2008R2.\n\tExperience in working with ADO.NET components such as Data grids, Datasets, Paging and Sorting.\n\tExtensively used Telerik controls for rich UI.\n\tImplemented object oriented concepts like Data Abstraction, Inheritance, Data Encapsulation and Polymorphism using C #.Net.\n\tHands on Experience on ASP.NET MVC patter

In [43]:
myresume_corpus = modify_text(myresume)

  0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
myresume_vector = count.transform(myresume_corpus)

In [45]:
myresume_vector = myresume_vector.toarray()

In [46]:
result1 = []
max_distance1 = 0
max_index1 = 0

for i in tqdm(range(len(vector_data))):
    result1.append([cosine_similarity(myresume_vector[0], vector_data[i]), i])
    if max_distance1 < cosine_similarity(resumes_vector[0], vector_data[i]):
        max_distance1 = cosine_similarity(resumes_vector[0], vector_data[i])
        max_index1 = i

  0%|          | 0/2413 [00:00<?, ?it/s]

In [47]:
print('This is job offer')
initial_data.loc[max_index1]['Description']

This is job offer


'C# .NET Developer\nUnited Kingdom - Remote Posted: 19/04/2023 \nSalary: ¬£50K per Year  ID: 32104_BH\n\n\n\nC# .NET Developer\nRemote\n\nUp to ¬£50,000\n\n We are seeking a talented and experienced C# .NET Developer to join our innovative client. The successful candidate will be responsible for designing, developing, and implementing software solutions using C# .NET programming language. \n \n If you are a proactive and enthusiastic C# .NET Developer looking for an exciting opportunity to work with an innovative company, this is the role for you! \n \n\nKey responsibilities:\n\n\nDevelop, design, and implement software solutions using C# .NET programming language \nParticipate in the full software development lifecycle, including requirements gathering, design, development, testing, and maintenance \nWrite clean, efficient, and well-documented code \nCollaborate with cross-functional teams to identify and solve software problems \n\n\nRequirements:\n\n\nProven experience as a C# .NET 

In [48]:
k = sorted(result1, reverse = True)

In [49]:
best_ind = []
for i in range(len(k)):
    if k[i][0]>0.1:
        best_ind.append(k[i][1])

In [50]:
# qq = data[data['Регион ']=='Акмолинская область']

In [51]:
ret = []
for i in best_ind:
    job = {
        "Position": data['Position'][i],
        "Company": data['Company'][i],
        "Link": data['Link'][i],
        "Description": data['Description'][i]
    }
    ret.append(job)

In [52]:
# def keyword_df(keyword, data_df):
#     keyword = keyword.lower()
#     keyword = keyword.split()
#     for i in range(len(data_df)):
#         data_df['Position'][i] = data_df['Position'][i].lower()
#         discard = True
#         for j in keyword:
#             if j in data_df['Position'][i]:
#                 discard = False
#         if discard:
#             data_df['Description'][i] = ''

#     return data_df

In [53]:
ret

[{'Position': 'IT Support Engineer x 2 (based at GCU London)',
  'Company': 'Glasgow Caledonian University',
  'Link': 'https://uk.indeed.com/applystart?jk=eb6add1bffa1a0b7&from=vj&pos=bottom&mvj=0&spon=0&sjdu=mcLJI0lO9xR5mAT-NkUfzYDDplm6SwWjHcxyoDIphKlYCv2PEgHDNSGLLQhoiVihkN0wkll9wDHc3mnStM4Hmg&vjfrom=serp&astse=987baa594c06ac8d&assa=7941',
  'Description': "Ref no: \n       \n\n        GCU01431\n       \n\n\n\n         Published: \n       \n\n        20/4/2023\n       \n\n\n\n         Closes: \n       \n\n        4/5/2023\n       \n\n\n\n\n\n\n\n\n\n\n\n\n\n        Location: \n       \n\n        London Campus, 40 Fashion Street, Spitalfields, E1 6PX\n       \n\n\n\n         Salary: \n       \n\n        ¬£30,619 - ¬£35,308 per year \n       \n\n\n\n        Contract Type: \n       \n\n        Permanent\n       \n\n\n\n         Position Type: \n       \n\n        Full Time\n       \n\n\n\n         Hours: \n       \n\n        35 hours per week\n       \n\n\n\n \n\n\nAdvert\n This full ti

In [54]:
data

,Position,Company,Link,Description
0,Technical Content Developer / writer,Korber Supply Chain,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,K√∂rber is the home for passionate people who ...
1,Software Engineer,Vodafone,https://uk.indeed.com/applystart?jk=df516aec25...,Software Engineer \n\n\nLocation: Blended / ho...
2,Software Support Lead,Acacium Group,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,CHS Healthcare (part of Acacium Group)\n Softw...
3,Backend Software Engineer,Enfuce,https://uk.indeed.com/applystart?jk=cef2e4b90b...,"ABOUT US\n Founded in 2016, Enfuce has become ..."
4,Graduate Software Developer,Townhouse,https://uk.indeed.com/rc/clk?jk=511e448b6c4c01...,About Townhouse\n Townhouse was created with a...
...,...,...,...,...
2408,Finance Analyst,PureGym Limited,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,The PureGym Group is the UKs largest gym opera...
2409,Data Protection Officer,BCT Resourcing,https://uk.indeed.com/applystart?jk=a6b9dec315...,We are looking for a Data Protection Officer t...
2410,P2P and Master Data Manager,Robert Half,https://uk.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Robert Half is partnering exclusively with one...
2411,Ecommerce Product Data Administrator Apprentice,QA Apprenticeships,https://uk.indeed.com/applystart?jk=975e1e47c6...,Employer description: ZLT Electrical are a fas...


In [55]:
data.to_csv('jobindeed.csv', index=False)